# Start the RIAK Server for the CICI SAFE vSDX

#### Configure the Envrionment

In [ ]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
export OS_PROJECT_NAME='CH-816532'

# Set region (again, no default currently)
export OS_REGION_NAME='CHI@TACC'

# Set chameleon keypair name and path to the private ssh key
export SSH_KEY_NAME=${USERNAME}-jupyter
export SSH_PRIVATE_KEY=${HOME}/work/pruth-chameleon-jupyter

# Set the reservations to use.  
# CONTROLLER_RESERVATION_ID can be for any type of node
# NODE_RESERVATION_ID must be for Skylake nodes
#export RIAK_RESERVATION_ID='8882c0ea-e92a-404e-8637-73b4c38f4101'


#Set the working directory
export WORKING_DIR=${HOME}/work/CICI-SAFE/Chameleon-Jupyter-Tridentcom

export UNIVERSAL_INFO_FILE=${WORKING_DIR}/universal_info
export RIAK_INFO_FILE=${WORKING_DIR}/riak_info
echo "" > $RIAK_INFO_FILE

#### Create Lease (if necessary)

In [ ]:
#Check of universal reservation
if [ -f "${UNIVERSAL_INFO_FILE}" ]; then
   LEASE_NAME=`cat $UNIVERSAL_INFO_FILE | grep LEASE_NAME | cut -d " " -f 2`
   echo $LEASE_NAME
   echo LEASE_NAME $LEASE_NAME >> $RIAK_INFO_FILE
fi

if  [ -z $LEASE_NAME ]; then
  
    echo No universal reservation. Creating new reservation
    # --start-date "2019-02-10 17:50" \
    # --end-date "2019-02-11 12:00" \
    START_DATE=`date -d "+2 min" +'%F %H:%M'`
    END_DATE=`date -d "+1 day" +'%F %H:%M'`
    LEASE_NAME=${USERNAME}"-riak-lease"
    echo LEASE_NAME $LEASE_NAME >> $RIAK_INFO_FILE

    NODE_MIN=1
    NODE_MAX=1

    NODE_TYPE='compute_haswell'
    #NODE_TYPE='compute_skylake'

    echo Creating Lease ${LEASE_NAME}
    blazar lease-create \
       --physical-reservation min=${NODE_MIN},max=${NODE_MAX},resource_properties='["=", "$node_type", "'${NODE_TYPE}'"]' \
       --start-date "${START_DATE}" \
       --end-date "${END_DATE}" \
       ${LEASE_NAME}

fi

echo LEASE_NAME: $LEASE_NAME
cat  $RIAK_INFO_FILE

In [ ]:
LEASE_NAME=`cat $RIAK_INFO_FILE | grep LEASE_NAME | cut -d " " -f 2`
RESERVATION_ID=`blazar lease-show --format value -c id ${LEASE_NAME}`
echo RESERVATION_ID $RESERVATION_ID

RESERVATION_RESOURCE_ID=`blazar lease-show --format value -c reservations ${RESERVATION_ID} | grep \"id\" | cut -d "\"" -f 4`
echo RESERVATION_RESOURCE_ID $RESERVATION_RESOURCE_ID

#blazar lease-show ${RESERVATION_ID}

while [[ true ]]
do 
	RESERVATION_STATUS=`blazar lease-show --format value -c status ${RESERVATION_ID}`
    
    if [[ "${RESERVATION_STATUS}" == "ACTIVE" ]]; then break; fi
    
    echo Reservation is ${RESERVATION_STATUS}. retrying in 5 seconds.
    sleep 5
done

echo RESERVATION_STATUS ${RESERVATION_STATUS}
echo RESERVATION_RESOURCE_ID $RESERVATION_RESOURCE_ID >> $RIAK_INFO_FILE

cat $RIAK_INFO_FILE
#blazar lease-show ${RESERVATION_ID}

#### Start RIAK Server

In [ ]:
#RESERVATION_RESOURCE_ID=`cat $RIAK_RESERVATION_FILE`
RESERVATION_RESOURCE_ID=`cat $RIAK_INFO_FILE | grep RESERVATION_RESOURCE_ID | cut -d " " -f 2`
echo  RESERVATION_RESOURCE_ID $RESERVATION_RESOURCE_ID

RIAK_NETWORK='sharednet1'
RIAK_NODE_NAME=${USERNAME}-riak
RIAK_STACK_NAME=${USERNAME}-riak

echo RIAK_NODE_NAME $RIAK_NODE_NAME >> $RIAK_INFO_FILE
echo RIAK_STACK_NAME $RIAK_STACK_NAME >> $RIAK_INFO_FILE

In [ ]:
echo Creating RIAK server. This will take several minutes!
openstack stack create  --max-width 80 \
        --template riak.yaml \
        --parameter "key_name=${SSH_KEY_NAME}" \
        --parameter "reservation_id=${RESERVATION_RESOURCE_ID}" \
        --parameter "network_name=${RIAK_NETWORK}" \
        --parameter "riak_node_name=${RIAK_NODE_NAME}" \
        --wait \
        ${RIAK_STACK_NAME}

In [ ]:
echo RIAK server creation complete! 
echo ${RIAK_NODE_NAME} status `openstack server show  --format value -c status ${RIAK_NODE_NAME}`

RIAK_PUBLIC_IP=`openstack server show  --format value -c addresses ${RIAK_NODE_NAME} | cut -d " " -f 2`
echo The RIAK se public IP is $RIAK_PUBLIC_IP
echo RIAK_PUBLIC_IP $RIAK_PUBLIC_IP >> $RIAK_INFO_FILE

In [ ]:
RIAK_PUBLIC_IP=`cat $RIAK_INFO_FILE | grep RIAK_PUBLIC_IP | cut -d " " -f 2`

#Poll for node to be ready for ssh
while [[ true ]]
do 
	ssh -q -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${RIAK_PUBLIC_IP} \
    exit
    
    if [[ "$?" == "0" ]]; then break; fi
    
    echo Node not ready for ssh. retrying in 5 seconds.
    sleep 5
done

echo RIAK node ready for ssh

#### Clean Up RIAK

In [ ]:
RIAK_STACK_NAME=`cat $RIAK_INFO_FILE | grep RIAK_STACK_NAME | cut -d " " -f 2`
openstack stack delete --wait -y $RIAK_STACK_NAME

LEASE_NAME=`cat $RIAK_INFO_FILE | grep LEASE_NAME | cut -d " " -f 2`
blazar lease-delete $LEASE_NAME

cat $RIAK_INFO_FILE
mv $RIAK_INFO_FILE ${RIAK_INFO_FILE}.old